In [94]:
import requests
from bs4 import BeautifulSoup
import csv

In [95]:
url = 'https://www.imdb.com/search/title/?sort=num_votes,desc&start=1&title_type=feature&year=1950,2012'

response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')
imdb_divs = soup.find_all('div', class_='lister-item-content')
stars = soup.find('p', class_='')

imdb_data = []

for imdb_div in imdb_divs:
    title = imdb_div.find('h3', class_='lister-item-header').find('a').get_text()
    director = imdb_div.find('p', class_='').find('a').get_text()
    stars_elements = imdb_div.find('p', class_='').find_all('a')
    stars = [star.get_text() for star in stars_elements]
    rating = imdb_div.find('div', class_='ratings-imdb-rating').find('strong').get_text()
    certificate = imdb_div.find('span', class_='certificate').get_text()
    runtime = imdb_div.find('span', class_='runtime').get_text()
    genre = imdb_div.find('span', class_='genre').get_text()
    gross = imdb_div.find('span', string='Gross:').find_next('span', {'name': 'nv'}).get_text()

    imdb_data.append({
        'title': title,
        'director': director,
        'stars': stars,
        'rating': rating,
        'certificate': certificate,
        'runtime': runtime,
        'genre': genre,
        'gross': gross
    })

In [58]:
csv_file = "imdb_data.csv"
csv_columns = [
    "title",
    "director",
    'stars',
    "rating",
    "certificate",
    "runtime",
    "genre",
    "gross",
]

with open(csv_file, "w", newline="", encoding="utf-8-sig") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for data in imdb_data:
        writer.writerow(data)

print(f"Data has been exported to {csv_file}")

Data has been exported to imdb_data.csv


In [12]:
import pandas as pd

df = pd.read_csv('imdb_data.csv')

In [10]:
director_movies_count = df.groupby('director').size().reset_index(name='Movie Count').sort_values(by=['Movie Count'], ascending=False)
director_movies_count.head(1)

,director,Movie Count
2,ChristopherNolan,6


In [13]:
df['genre'] = df['genre'].str.split(',').apply(lambda x: [genre.strip() for genre in x])

split_genres = []

for genres_list in df['genre']:
    split_genres.extend(genres_list)

# creating a dataframe for separate genres
genre_count_df = pd.DataFrame({'genre': split_genres})

genre_movies_count = genre_count_df.groupby('genre').size().reset_index(name='Movie Count').sort_values(by=['Movie Count'], ascending=False)

genre_movies_count


,genre,Movie Count
6,Drama,33
0,Action,20
1,Adventure,17
5,Crime,14
13,Sci-Fi,8
14,Thriller,6
8,Fantasy,6
11,Mystery,5
4,Comedy,4
2,Animation,4
